In [1]:
import numpy as np
num_data_points = 20
num_dimensions = 10
data = np.loadtxt('data_HW8_Q4.csv', delimiter=',')
num_clusters = 3
#dfmax, dfmin = data.max(), data.min()
#data = 1*(data - dfmin)/(dfmax - dfmin)
data_normed = (data - data.min(0)) / data.ptp(0) 
data = data_normed

In [2]:
from pyomo.environ import *
from pyomo import *
import pandas as pd
import math
from numpy import inf
import heapq
import logging
from pyomo.environ import *
max_d2 = 0
for i in range(0, num_data_points):
    for j in range(0, num_data_points):
        max_d2 = max(max_d2,sum((data[i][k]-data[j][k])**2 for k in range(0,num_dimensions)))
print(max_d2)
        
# Set the logging level to ERROR to suppress infeasibility warnings
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
model = ConcreteModel()

# Set declarations 
model.I = Set(initialize=range(1, num_data_points+1))  
model.K = Set(initialize=range(1, num_clusters+1))
model.D = Set(initialize=range(1, num_dimensions+1))
model.d = Var(model.I,domain = NonNegativeReals)
model.y = Var(model.I,model.K, domain = Binary)
model.c = Var(model.K,model.D)
model.M = max_d2*10
def init_func(m, idx1,idx2):
    return data[idx1-1][idx2-1]
model.p = Param(model.I, model.D, initialize = init_func)
fac = 1
model.con1 =  Constraint(model.I,model.K, rule=lambda model, i,k: 
        fac*model.d[i]*1 >= fac*sum((model.p[i,j]-model.c[k,j])**2 for j in model.D)-fac*model.M*(1-model.y[i,k]))
model.con2 =  Constraint(model.I, rule=lambda model, i: 
        sum(model.y[i,k] for k in model.K) ==1)

model.cost = Objective(sense=minimize, expr = sum(model.d[i] for i in model.I))
solver = SolverFactory('gurobi')
solver.options['MIPGap'] = 0.01
solver.options['TimeLimit'] = 300
solver.options['Threads'] = 8
results = solver.solve(model,tee = True)

5.060866316636287
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file C:\Users\aramanuj\AppData\Local\Temp\tmp1kobnsnd.pyomo.lp
Reading time = 0.00 seconds
x1: 20 rows, 110 columns, 60 nonzeros
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 300
Set parameter Threads to value 8
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 110 columns and 60 nonzeros
Model fingerprint: 0x0d541533
Model has 60 quadratic constraints
Variable types: 50 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-02, 5e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
 

In [3]:
clus = {}
clus_num = {1:0,2:0,3:0}
for i in model.I:
    for k in model.K:
        if(model.y[i,k]()==1):
            clus[i] = k
            clus_num[k] = clus_num[k]+1
print(clus)
print(clus_num)

{1: 1, 2: 1, 3: 2, 4: 2, 5: 1, 6: 1, 7: 2, 8: 2, 9: 1, 10: 3, 11: 3, 12: 1, 13: 1, 14: 3, 15: 3, 16: 3, 17: 2, 18: 2, 19: 2, 20: 3}
{1: 7, 2: 7, 3: 6}


In [10]:
import numpy as np
num_data_points = 20
num_dimensions = 3
data = np.loadtxt('data_HW8_Q4.csv', delimiter=',')
num_clusters = 3
#dfmax, dfmin = data.max(), data.min()
#data = 1*(data - dfmin)/(dfmax - dfmin)
data_normed = (data - data.min(0)) / data.ptp(0) 
data = data_normed[:,0:num_dimensions]
print(data.shape)

(20, 3)


In [11]:
from pyomo.environ import *
from pyomo import *
import pandas as pd
import math
from numpy import inf
import heapq
import logging
from pyomo.environ import *
max_d2 = 0
for i in range(0, num_data_points):
    for j in range(0, num_data_points):
        max_d2 = max(max_d2,sum((data[i][k]-data[j][k])**2 for k in range(0,num_dimensions)))
print(max_d2)
        
# Set the logging level to ERROR to suppress infeasibility warnings
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
model = ConcreteModel()

# Set declarations 
model.I = Set(initialize=range(1, num_data_points+1))  
model.K = Set(initialize=range(1, num_clusters+1))
model.D = Set(initialize=range(1, num_dimensions+1))
model.d = Var(model.I,domain = NonNegativeReals)
model.y = Var(model.I,model.K, domain = Binary)
model.c = Var(model.K,model.D)
model.M = max_d2*10
def init_func(m, idx1,idx2):
    return data[idx1-1][idx2-1]
model.p = Param(model.I, model.D, initialize = init_func)
fac = 1
model.con1 =  Constraint(model.I,model.K, rule=lambda model, i,k: 
        fac*model.d[i]*1 >= fac*sum((model.p[i,j]-model.c[k,j])**2 for j in model.D)-fac*model.M*(1-model.y[i,k]))
model.con2 =  Constraint(model.I, rule=lambda model, i: 
        sum(model.y[i,k] for k in model.K) ==1)

model.cost = Objective(sense=minimize, expr = sum(model.d[i] for i in model.I))
solver = SolverFactory('gurobi')
solver.options['MIPGap'] = 0.01
solver.options['TimeLimit'] = 300
solver.options['Threads'] = 8
results = solver.solve(model,tee = True)

1.5459880283099685
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-04
Read LP format model from file C:\Users\aramanuj\AppData\Local\Temp\tmptj56kckf.pyomo.lp
Reading time = 0.00 seconds
x1: 20 rows, 89 columns, 60 nonzeros
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 300
Set parameter Threads to value 8
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 89 columns and 60 nonzeros
Model fingerprint: 0xcdc1be25
Model has 60 quadratic constraints
Variable types: 29 continuous, 60 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [8e-02, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  

In [12]:
clus = {}
clus_num = {1:0,2:0,3:0}
for i in model.I:
    for k in model.K:
        if(model.y[i,k]()==1):
            clus[i] = k
            clus_num[k] = clus_num[k]+1
print(clus)
print(clus_num)

{1: 2, 2: 3, 3: 1, 4: 1, 5: 2, 6: 3, 7: 2, 8: 1, 9: 3, 10: 3, 11: 3, 12: 3, 13: 3, 14: 1, 15: 3, 16: 2, 17: 2, 18: 1, 19: 2, 20: 1}
{1: 6, 2: 6, 3: 8}
